In [1]:
import sys
sys.path.append('../../')

In [2]:
import data.loader
(train, train_y), (test, test_y), (_, _), one_hot_encoder_X, one_hot_encoder_y = data.loader.encode_and_partition(
    data.loader.diabetic_data_raw
)

In [3]:
import wandb
import os

# "ae_encoder_run_path": "charles-s-strauss-n-a/ae/runs/l5uu22fm",
ae_encoder_run_path = "charles-s-strauss-n-a/ae/runs/ugoaq6gb"

In [5]:
aeweights = wandb.restore(
    'autoencoder.pth',
    run_path=ae_encoder_run_path
)

import importlib.util

with wandb.restore('files/model.py',
    run_path=ae_encoder_run_path) as modelpy:
    
    spec = importlib.util.spec_from_loader(modelpy.name, loader=None)
    aemodel = importlib.util.module_from_spec(spec)
    exec(modelpy.read(), aemodel.__dict__)

aemodel

<module '/scratch/u1319464/BMI6015-diabetes-final/models/ae/files/model.py'>

In [12]:
input_dim = train.shape[1]
ae = aemodel.AutoEncoder(input_dim=input_dim, hidden_dim=128//4, latent_dim=64//4)
ae.load_state_dict(torch.load(aeweights.name))

<All keys matched successfully>

In [8]:
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64

transform = transforms.Compose([transforms.ToTensor()])

train_data = torch.from_numpy(train.toarray().astype(np.float32))
train_dataset = TensorDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

test_data = torch.from_numpy(test.toarray().astype(np.float32))
test_dataset = TensorDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

ae.eval()
with torch.no_grad():
    latent_representations = ae.encoder(torch.tensor(train_data)).numpy()

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
latent_2d = tsne.fit_transform(latent_representations)



/tmp/ipykernel_3104455/4112939000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  latent_representations = ae.encoder(torch.tensor(train_data)).numpy()


In [ ]:

# Assuming 'train_labels' contains your labels
plt.figure(figsize=(10, 8))
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=train_labels, cmap='viridis', s=5)
plt.colorbar(scatter, label='Labels')
plt.title("t-SNE Visualization of Latent Representations")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show()